# Functionality 1 - Find the Neighbours!

Take input:

a node v

One of the following distances function: 

t(x,y), d(x,y) or network distance .

a distance threshold d.

Implement an algorithm (using proper data structures) that returns :

the set of nodes at distance <= d from v, corresponding to v’s neighborhood.

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Read DS

# Distance :

In [2]:
df_D = pd.read_csv('./DATA/Distance_graph',header=0, sep=',',
                index_col=None, encoding = "ISO-8859-1")
df_D.columns=["Distance"]
df_D.drop([0,1,2,3,4,5],inplace=True)
df_D.reset_index(drop=True)
df_D.head(10)


,Distance
6,a 1 1048577 456
7,a 1048577 1 456
8,a 2 1048578 2389
9,a 1048578 2 2389
10,a 3 1048579 358
11,a 1048579 3 358
12,a 4 1048579 14901
13,a 1048579 4 14901
14,a 5 6 18133
15,a 6 5 18133


In [3]:
df_D=df_D['Distance'].str.split(" " , expand=True)
df_D.columns=['a','Node1','Node2','Distance']
df_D['Node1']=pd.to_numeric(df_D['Node1'])
df_D['Node2']=pd.to_numeric(df_D['Node2'])
df_D['Distance']=pd.to_numeric(df_D['Distance'])
df_D=df_D.reset_index(drop=True)
df_D.head(4)

,a,Node1,Node2,Distance
0,a,1,1048577,456
1,a,1048577,1,456
2,a,2,1048578,2389
3,a,1048578,2,2389


In [4]:
df_D.shape

(4657742, 4)

In [5]:
df_T = pd.read_csv('./DATA/Travel_time_graph',header=0, sep=',',
                index_col=None, encoding = "ISO-8859-1")
df_T.columns=["Distance"]
df_T.drop([0,1,2,3,4,5],inplace=True)
df_T.head(5)

,Distance
6,a 1 1048577 1139
7,a 1048577 1 1139
8,a 2 1048578 5972
9,a 1048578 2 5972
10,a 3 1048579 895


In [6]:
df_T=df_T["Distance"].str.split(" " , expand=True)
df_T.columns=['a','Node1','Node2','Distance']
df_T['Node1']=pd.to_numeric(df_T['Node1'])
df_T['Node2']=pd.to_numeric(df_T['Node2'])
df_T['Distance']=pd.to_numeric(df_T['Distance'])
df_T=df_T.reset_index(drop=True)
df_T.head(5)

,a,Node1,Node2,Distance
0,a,1,1048577,1139
1,a,1048577,1,1139
2,a,2,1048578,5972
3,a,1048578,2,5972
4,a,3,1048579,895


In [7]:
df_T.shape

(4657742, 4)

# Search Neighbours:


In [8]:
node= int(input("Enter node numbner "))
distance_type=input("Enter one of this Distance type: Time or Physical ")
threshold=int(input("Enter your threshold "))  
#The idea is to iterate all direct neighbour from node v
#and check the values Corresponding to the input
#and we recursively subtract the new distance from initial threshold 
#to check the neighbours of the initial node's neighbour's Satisfying the threshold
def search_Neighbours(node, distance_type, threshold): 
    if distance_type == "Physical":
        data = df_D
    elif distance_type == "Time":
        data=df_T   
    out_df = data[(data['Node1'] == node) & (data['Distance'] <= threshold)]

    Neighbours=list(out_df.Node2.values)
        #print(Neighbours)
    for j in Neighbours:
        neigh_dis = data[(data['Node1'] == node) & (data['Node2'] == j)]
        neigh_dis=int(neigh_dis['Distance'])
        out_df=pd.concat([out_df,search_Neighbours(j, distance_type, threshold-neigh_dis)])
    return out_df

out_df=search_Neighbours(node,distance_type,threshold)
out_df.head(7)

Enter node numbner 300
Enter one of this Distance type: Time or Physical Physical
Enter your threshold 5000


,a,Node1,Node2,Distance
1003,a,300,1048816,1018
1008,a,300,1048818,597
1002,a,1048816,300,1018
1071,a,1048816,1048832,365
1003,a,300,1048816,1018
1008,a,300,1048818,597
1002,a,1048816,300,1018


In [9]:
all_Neighbours = list(set(out_df['Node2'].values))
all_Neighbours

[1048832,
 228,
 1048836,
 300,
 303,
 1048816,
 2448,
 1048818,
 1048819,
 1048817,
 1048820]

# Visualization 1 - Visualize the Neighbours!
#### The map 

In [10]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [11]:
df_c = pd.read_csv('./DATA/Coordinates',header=0, sep=',',
                index_col=None, encoding = "ISO-8859-1")
df_c.columns=["Coordinates"]
df_c.drop([0,1,2,3,4,5],inplace=True)
df_c.head(3)

,Coordinates
6,v 1 -114315309 34133550
7,v 2 -114223946 34176221
8,v 3 -114307299 34148791


In [12]:
df_c=df_c.Coordinates.str.split(" " , expand=True)
df_c.columns=['v','Node_ID','long','lat']
df_c['Node_ID']=pd.to_numeric(df_c['Node_ID'])
df_c['long']=pd.to_numeric(df_c['long'])
df_c['lat']=pd.to_numeric(df_c['lat'])
df_c=df_c.reset_index(drop=True)
df_c.head(5)

,v,Node_ID,long,lat
0,v,1,-114315309,34133550
1,v,2,-114223946,34176221
2,v,3,-114307299,34148791
3,v,4,-114318765,34138889
4,v,5,-114347300,34042614


In [13]:
df_c['lat']=df_c['lat']/1000000
df_c['long']=df_c['long']/1000000

In [14]:
df_c.head(4)

,v,Node_ID,long,lat
0,v,1,-114.315309,34.133550
1,v,2,-114.223946,34.176221
2,v,3,-114.307299,34.148791
3,v,4,-114.318765,34.138889


In [15]:
out_df_cor=pd.merge(df_c,out_df, right_on="Node2", left_on='Node_ID' )
out_df_cor.head(5)

,v,Node_ID,long,lat,a,Node1,Node2,Distance
0,v,228,-114.033229,33.953738,a,1048832,228,1957
1,v,228,-114.033229,33.953738,a,1048832,228,1957
2,v,228,-114.033229,33.953738,a,1048832,228,1957
3,v,228,-114.033229,33.953738,a,1048832,228,1957
4,v,300,-114.030468,33.955685,a,1048816,300,1018


In [16]:
out_df_cor=out_df_cor.drop_duplicates()

In [17]:
out_df_cor.head(5)

,v,Node_ID,long,lat,a,Node1,Node2,Distance
0,v,228,-114.033229,33.953738,a,1048832,228,1957
4,v,300,-114.030468,33.955685,a,1048816,300,1018
7,v,300,-114.030468,33.955685,a,1048818,300,597
31,v,303,-114.029393,33.956453,a,1048818,303,711
45,v,303,-114.029393,33.956453,a,1048819,303,1577


In [18]:
lat=list(df_c['lat'].values)
long=list(df_c['long'].values)
m = folium.Map(location=[np.mean(lat),np.mean(long)],zoom_start=6)

In [19]:
m

In [20]:
# plotting the starting node and node within the threshold
for i in range(len(out_df_cor)):
    folium.CircleMarker(location=[lat[i],long[i]],
                        radius=5,
                        popup=['Node_ID'],
                        fill_color="#3db7e4",
                       ).add_to(m)

In [21]:
m.save('index.html')
m

# Functionalities 3 - 4

Import the libraries we will need

In [1]:
import collections
import math
import gmplot
import statistics
import itertools
import func_3_4 as f

### Functionality 3

This is the function to call to use funtionality 3.

In [9]:
def functionality_3():
    # Call the setup function to initialize our elements. 
    # Sequence = nodes to visit, graph = the graph of all the nodes and edges, mode = distance, time, network distance
    sequence, graph, mode = f.setup()    
    # Calculate the shortest path for each subpath and concatenate them 
    # If the mode = nd we need to pass to the function walk "nd = True"
    path = []
    if mode == "nd":
        for i in range(len(sequence)-1):
            path = path + f.walk(graph, sequence[i], sequence[i+1], nd = True)[:-1]
        print(path)
    else:
        for i in range(len(sequence)-1):
            path = path + f.walk(graph, sequence[i], sequence[i+1])[:-1]
        print(path+[sequence[-1]])
        
    # Visualization
    # Store the coordinate for each point we visited in a dictionary ID:[longitude, latitude]
    coordinate = {}
    with open(r"C:\Users\39335\Desktop\University\Aris\USA-road-d.CAL.co") as file:
        for line in file:
            if line.startswith("v"):
                l = [int(x) for x in line[2:].split()]
                if l[0] not in coordinate:
                    coordinate[l[0]] = [l[1], l[2]]
                    
    # Store in two different lists the coordinates of the points we visited
    latitude_list = []
    longitude_list = []
    for i in range(len(path)):
        latitude_list.append((coordinate[path[i]][1])/10**6)    #The division by 10^6 is to adjust the coordinates in the file to the real world
        longitude_list.append((coordinate[path[i]][0])/10**6)
    
    # Set the center of the map using the mean of the coordinates of the points in our graph
    mid_lon = statistics.mean(longitude_list)
    mid_lat = statistics.mean(latitude_list)
    gmap = gmplot.GoogleMapPlotter(mid_lat, mid_lon, 12.5) 
    
    # Plot using gmap
    gmap.scatter( latitude_list, longitude_list, '# FF0000', size = 40, marker = False ) 
    gmap.plot(latitude_list, longitude_list, 'red', edge_width = 2.5) 
    # Set the directory we want to output the html file
    gmap.draw(r"C:\Users\39335\Desktop\University\Aris\map3.html")
    

### Functionality 4

To find the shortest path passing through the given nodes we will need to calculate the distance for all the permutation of the possible paths. Then we choose the one with the lesser weight. <br>
The rest of the code is a lot similar to functionality 3. 

In [3]:
def functionality_4():
    sequence, graph, mode = f.setup()  
    
    # Permutation of all the nodes that are not the starting nor the final one. In this way we check every possible route.
    perm = list(itertools.permutations(sequence[1:-1]))
    min_weight = math.inf
    min_path = []
    
    # For every permutation we compute the weight and we check if it is the lowest.
    # In that case we update the min path and the min weight
    if mode == "nd":
        for p in perm:
            current_path = []
            current_weight = 0
            for i in range(len(p)-1):
                current_path = current_path + f.walk(graph, p[i], p[i+1], nd = True)[:-1]
                current_weight = current_weight + f.walk(graph, p[i], p[i+1], nd = True, w = True)
            if (current_weight < min_weight):
                min_weight = current_weight 
                min_path = current_path
        min_path = min_path + [sequence[-2]]
        # Concatenate the path that goes from starting point (H) to p1, and the one that goes from p_n-1 to p_n. (H and p_n are fixed)
        path = f.walk(graph, sequence[0], min_path[0], nd = True)[:-1] + min_path + f.walk(graph, min_path[-2], sequence[-1], nd = True)
        print(path)
        
    else:
        for p in perm:
            current_path = []
            current_weight = 0
            for i in range(len(p)-1):
                current_path = current_path + f.walk(graph, p[i], p[i+1])[:-1]
                current_weight = current_weight + f.walk(graph, p[i], p[i+1], w = True)
            if (current_weight < min_weight):
                min_weight = current_weight 
                min_path = current_path
        min_path = min_path + [sequence[-2]]
        path = f.walk(graph, sequence[0], min_path[0])[:-1] + min_path + f.walk(graph, min_path[-2], sequence[-1])
        print(path)
    
    
    
    # Visualization
    coordinate = {}
    with open(r"C:\Users\39335\Desktop\University\Aris\USA-road-d.CAL.co") as file:
        for line in file:
            if line.startswith("v"):
                l = [int(x) for x in line[2:].split()]
                if l[0] not in coordinate:
                    coordinate[l[0]] = [l[1], l[2]]
                    
    latitude_list = []
    longitude_list = []
    for i in range(len(path)):
        latitude_list.append((coordinate[path[i]][1])/10**6)
        longitude_list.append((coordinate[path[i]][0])/10**6)
    
    mid_lon = statistics.mean(longitude_list)
    mid_lat = statistics.mean(latitude_list)
    gmap = gmplot.GoogleMapPlotter(mid_lat, mid_lon, 12.5) 
    
    gmap.scatter( latitude_list, longitude_list, '# FF0000', size = 40, marker = False ) 
    gmap.plot(latitude_list, longitude_list, 'magenta', edge_width = 2.5) 
    gmap.draw(r"C:\Users\39335\Desktop\University\Aris\map4.html")

In [10]:
functionality_3()

Write a node H, the sequence of nodes to visit in order p_1,...,p_n, and the mode you want t(time) / d(distance) / nd(network distance). Ex. '25 50,22,13,4 t'
1 20,40,50,60 nd
[1, 1803, 1802, 1050022, 1808, 1050024, 1810, 1050026, 1813, 1050030, 2199, 1050339, 1050261, 2102, 2092, 2093, 1050262, 2078, 1050243, 1050266, 2118, 2125, 1050280, 2122, 1050277, 2142, 1050291, 2522, 2524, 1050447, 2153, 2152, 2154, 2158, 1050306, 1050308, 2159, 34, 1048603, 1048598, 28, 1050699, 1050700, 1048601, 31, 1050698, 1048627, 64, 2655, 2656, 2657, 1048626, 179, 180, 35, 1048604, 1048606, 2662, 1050717, 2661, 40, 1048609, 1050708, 43, 1048612, 2384, 1048594, 23, 24, 1048595, 1048596, 21, 20, 21, 1048596, 1048595, 24, 23, 1048594, 2384, 1048612, 43, 1050708, 1048609, 40, 1048609, 2663, 41, 1048610, 42, 2670, 45, 44, 1048613, 1048650, 71, 1048634, 1048653, 103, 1048658, 104, 111, 114, 130, 133, 1048680, 132, 1048695, 152, 160, 1048703, 161, 1048710, 172, 1048848, 337, 338, 341, 367, 344, 1048851, 1048852

In [6]:
functionality_4()

Write a node H, the sequence of nodes to visit in order p_1,...,p_n, and the mode you want t(time) / d(distance) / nd(network distance). Ex. '25 50,22,13,4 t'
1 20,40,50,60 nd
[1, 1803, 1802, 1050022, 1799, 1050018, 1050017, 1800, 1050019, 1823, 451, 1048937, 1707, 1705, 1050005, 1778, 1050000, 1774, 1049998, 1050002, 1781, 1783, 1796, 1050015, 1795, 1050014, 1050383, 2249, 2248, 1049494, 1050385, 211, 1048724, 185, 184, 1048719, 183, 1048718, 382, 1048878, 376, 1048879, 377, 379, 1048880, 378, 1048881, 372, 371, 1048892, 393, 1048882, 380, 1048891, 1048875, 373, 392, 1049055, 1049056, 1049057, 601, 1049063, 612, 1049065, 614, 1049069, 1049074, 621, 1049085, 119, 1049087, 638, 645, 1050424, 2313, 1050423, 1049217, 1049094, 647, 640, 1049089, 1049090, 1049099, 652, 1049098, 2701, 800, 1049117, 798, 690, 691, 1049130, 1049131, 707, 715, 1049149, 718, 1049198, 774, 1049197, 780, 1048616, 49, 50, 49, 1048616, 780, 1049197, 774, 1049198, 718, 1049149, 715, 1049166, 721, 1049154, 725, 814, 5